# Tworzymy model do klastrowania danych z ankiety powitalnej

In [2]:
import pandas as pd
from pycaret.clustering import setup, create_model, assign_model, plot_model, save_model, load_model, predict_model

In [3]:
df = pd.read_csv('welcome_survey_simple_v2.csv', sep=';')
df.head()

,age,edu_level,fav_animals,fav_place,gender
0,<18,Podstawowe,Brak ulubionych,NaN,Kobieta
1,25-34,Średnie,Psy,Nad wodą,Mężczyzna
2,45-54,Wyższe,Psy,W lesie,Mężczyzna
3,35-44,Średnie,Koty,W górach,Mężczyzna
4,35-44,Wyższe,Psy,Nad wodą,Mężczyzna


In [5]:
df.describe()

,age,edu_level,fav_animals,fav_place,gender
count,229,229,229,200,227
unique,8,3,5,4,2
top,35-44,Wyższe,Psy,Nad wodą,Mężczyzna
freq,83,184,123,73,171


In [6]:
s = setup(df, session_id=123)
s

,Description,Value
0,Session id,123
1,Original data shape,"(229, 5)"
2,Transformed data shape,"(229, 21)"
3,Categorical features,5
4,Rows with missing values,13.1%
5,Preprocess,True
6,Imputation type,simple
7,Numeric imputation,mean
8,Categorical imputation,mode
9,Maximum one-hot encoding,-1


In [6]:
#stary setup dla porównania tylko zostawiam
#
#NIE URUCHAMIAĆ
#
s = setup(df, session_id=123)
s

,Description,Value
0,Session id,123
1,Original data shape,"(140, 5)"
2,Transformed data shape,"(140, 21)"
3,Categorical features,5
4,Rows with missing values,11.4%
5,Preprocess,True
6,Imputation type,simple
7,Numeric imputation,mean
8,Categorical imputation,mode
9,Maximum one-hot encoding,-1


In [7]:
s.dataset.head()

,age,edu_level,fav_animals,fav_place,gender
0,<18,Podstawowe,Brak ulubionych,NaN,Kobieta
1,25-34,Średnie,Psy,Nad wodą,Mężczyzna
2,45-54,Wyższe,Psy,W lesie,Mężczyzna
3,35-44,Średnie,Koty,W górach,Mężczyzna
4,35-44,Wyższe,Psy,Nad wodą,Mężczyzna


In [8]:
s.dataset_transformed.head()

,age_<18,age_25-34,age_45-54,age_35-44,age_18-24,age_>=65,age_55-64,age_unknown,edu_level_Podstawowe,edu_level_Średnie,...,fav_animals_Brak ulubionych,fav_animals_Psy,fav_animals_Koty,fav_animals_Inne,fav_animals_Koty i Psy,fav_place_Nad wodą,fav_place_W lesie,fav_place_W górach,fav_place_Inne,gender
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [9]:
kmeans = create_model('kmeans', num_clusters=8)

,Silhouette,Calinski-Harabasz,Davies-Bouldin,Homogeneity,Rand Index,Completeness
0,0.1925,27.1789,1.8733,0,0,0


Processing:   0%|          | 0/3 [00:00<?, ?it/s]

In [10]:
df_with_clusters = assign_model(kmeans)
df_with_clusters

,age,edu_level,fav_animals,fav_place,gender,Cluster
0,<18,Podstawowe,Brak ulubionych,NaN,Kobieta,Cluster 7
1,25-34,Średnie,Psy,Nad wodą,Mężczyzna,Cluster 7
2,45-54,Wyższe,Psy,W lesie,Mężczyzna,Cluster 3
3,35-44,Średnie,Koty,W górach,Mężczyzna,Cluster 4
4,35-44,Wyższe,Psy,Nad wodą,Mężczyzna,Cluster 0
...,...,...,...,...,...,...
224,35-44,Wyższe,Koty,Inne,Kobieta,Cluster 2
225,45-54,Wyższe,Inne,W lesie,Mężczyzna,Cluster 3
226,25-34,Wyższe,Psy,W górach,Mężczyzna,Cluster 1
227,35-44,Wyższe,Brak ulubionych,W górach,Mężczyzna,Cluster 2


In [11]:
df_with_clusters["Cluster"].value_counts()

Cluster
Cluster 3    38
Cluster 1    34
Cluster 0    30
Cluster 6    30
Cluster 5    29
Cluster 4    26
Cluster 2    23
Cluster 7    19
Name: count, dtype: int64

In [12]:
plot_model(kmeans, plot='cluster')

In [13]:
save_model(kmeans, 'welcome_survey_clustering_pipeline_v2', verbose=False)

(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=[], transformer=SimpleImputer())),
                 ('categorical_imputer',
                  TransformerWrapper(include=['age', 'edu_level', 'fav_animals',
                                              'fav_place', 'gender'],
                                     transformer=SimpleImputer(strategy='most_frequent'))),
                 ('ordinal_encoding',
                  TransformerWrapper(include=['gender'],
                                     transfo...
                                                                mapping=[{'col': 'gender',
                                                                          'data_type': dtype('O'),
                                                                          'mapping': Kobieta      0
 Mężczyzna    1
 NaN         -1
 dtype: int64}]))),
                 ('onehot_encoding',
                  TransformerWrappe

In [14]:
kmeans_pipeline = load_model('welcome_survey_clustering_pipeline_v2')
kmeans_pipeline

Transformation Pipeline and Model Successfully Loaded


Pipeline(memory=FastMemory(location=C:\Users\piotr\AppData\Local\Temp\joblib),
         steps=[('numerical_imputer',
                 TransformerWrapper(include=[], transformer=SimpleImputer())),
                ('categorical_imputer',
                 TransformerWrapper(include=['age', 'edu_level', 'fav_animals',
                                             'fav_place', 'gender'],
                                    transformer=SimpleImputer(strategy='most_frequent'))),
                ('ordinal_encoding',
                 Trans...
                                                               mapping=[{'col': 'gender',
                                                                         'data_type': dtype('O'),
                                                                         'mapping': Kobieta      0
Mężczyzna    1
NaN         -1
dtype: int64}]))),
                ('onehot_encoding',
                 TransformerWrapper(include=['age', 'edu_level', 'fav_animals',
                                             'fav_place'],
                                    transformer=OneHotEncoder(cols=['age',
                                                                    'edu_level',
                                                                    'fav_animals',
                                                                    'fav_place'],
                                                              handle_missing='return_nan',
                                                              use_cat_names=True))),
                ('trained_model', KMeans(random_state=123))])

In [15]:
predict_df = pd.DataFrame([
    {
        "age": "45-54",  # '<18', '25-34', '45-54', '35-44', '18-24', '>=65', '55-64', 'unknown'
        "edu_level": 'Średnie',  # 'Podstawowe', 'Średnie', 'Wyższe'
        "fav_animals": 'Brak ulubionych',  # 'Brak ulubionych', 'Psy', 'Koty', 'Inne', 'Koty i Psy'
        "fav_place": 'W lesie',  # 'Nad wodą', 'W lesie', 'W górach', 'Inne'
        "gender": 'Kobieta',  # 'Mężczyzna', 'Kobieta'
    }
])

In [16]:
predict_with_clusters_df = predict_model(kmeans_pipeline, data=predict_df)
predict_with_clusters_df["Cluster"]

0    Cluster 4
Name: Cluster, dtype: object